In [1]:
print("Starting the parsing program...")

Starting the parsing program...


In [37]:
!pip install requests psycopg2 python-dotenv

In [62]:
import requests
import json
import psycopg2
import os
import dotenv
from datetime import datetime

In [20]:
BASE_URL = "https://v3.football.api-sports.io"

url_leagues = f"{BASE_URL}/leagues"
url_fixtures = f"{BASE_URL}/fixtures"

headers = {
  'x-rapidapi-key': '316dede52d52efb30d6dcc7cfb2e936b',
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

In [ ]:
payload={}

response = requests.request("GET", url_leagues, headers=headers, data=payload)

In [ ]:
with open("leagues_response.json", "w", encoding="utf-8") as f:
    json.dump(response.json(), f, ensure_ascii=False, indent=4)

In [ ]:
with open("leagues_response.json", "r", encoding="utf-8") as f:
    data_league = json.load(f)

In [ ]:
for league in data_league['response']:
    if league['country']['name'] in ['England']:
        print(f"League id = {league['league']['id']}: {league['league']['name']}, Country: {league['country']['name']}")

League id = 39: Premier League, Country: England
League id = 40: Championship, Country: England
League id = 46: EFL Trophy, Country: England
League id = 45: FA Cup, Country: England
League id = 47: FA Trophy, Country: England
League id = 48: League Cup, Country: England
League id = 43: National League, Country: England
League id = 41: League One, Country: England
League id = 42: League Two, Country: England
League id = 528: Community Shield, Country: England
League id = 44: FA WSL, Country: England
League id = 49: National League - Play-offs, Country: England
League id = 698: FA Women's Cup, Country: England
League id = 50: National League - North, Country: England
League id = 51: National League - South, Country: England
League id = 52: Non League Div One - Isthmian North, Country: England
League id = 53: Non League Div One - Isthmian South Central, Country: England
League id = 54: Non League Div One - Northern West, Country: England
League id = 55: Non League Div One - Northern Midla

In [ ]:
def get_all_fixtures(league_id: int, season: int):
    page = 1
    fixtures = []

    while True:
        params = {
            "league": league_id,
            "season": season
        }
        response = requests.get(url_fixtures, headers=headers, params=params)
        data = response.json()

        if "response" not in data or not data["response"]:
            break

        fixtures.extend(data["response"])
        
        # Проверяем, есть ли следующая страница
        paging = data.get("paging", {})
        if paging.get("current") == paging.get("total"):
            break

        page += 1

    return fixtures

In [27]:
params = {
    "league": 39,
    "season": 2023
}

response = requests.get(url_fixtures, headers=headers, params=params)

In [29]:
with open("fixtures.json", "w", encoding="utf-8") as f:
    json.dump(response.json(), f, ensure_ascii=False, indent=2)

In [30]:
with open("fixtures.json", "r", encoding="utf-8") as f:
    fixtures = json.load(f)

In [33]:
for fixture in fixtures['response']:
    print(f"Fixture ID: {fixture['fixture']['id']}, Date: {fixture['fixture']['date']}, Teams: {fixture['teams']['home']['name']} vs {fixture['teams']['away']['name']}, Score: {fixture['goals']['home']} - {fixture['goals']['away']}")

Fixture ID: 1035037, Date: 2023-08-11T19:00:00+00:00, Teams: Burnley vs Manchester City, Score: 0 - 3
Fixture ID: 1035038, Date: 2023-08-12T11:30:00+00:00, Teams: Arsenal vs Nottingham Forest, Score: 2 - 1
Fixture ID: 1035039, Date: 2023-08-12T14:00:00+00:00, Teams: Bournemouth vs West Ham, Score: 1 - 1
Fixture ID: 1035041, Date: 2023-08-12T14:00:00+00:00, Teams: Everton vs Fulham, Score: 0 - 1
Fixture ID: 1035040, Date: 2023-08-12T14:00:00+00:00, Teams: Brighton vs Luton, Score: 4 - 1
Fixture ID: 1035042, Date: 2023-08-12T14:00:00+00:00, Teams: Sheffield Utd vs Crystal Palace, Score: 0 - 1
Fixture ID: 1035043, Date: 2023-08-12T16:30:00+00:00, Teams: Newcastle vs Aston Villa, Score: 5 - 1
Fixture ID: 1035044, Date: 2023-08-13T13:00:00+00:00, Teams: Brentford vs Tottenham, Score: 2 - 2
Fixture ID: 1035045, Date: 2023-08-13T15:30:00+00:00, Teams: Chelsea vs Liverpool, Score: 1 - 1
Fixture ID: 1035046, Date: 2023-08-14T19:00:00+00:00, Teams: Manchester United vs Wolves, Score: 1 - 0
Fixtu

In [ ]:
# Пример: Английская Премьер-лига (league_id=39), сезон 2023
fixtures = get_all_fixtures(league_id=39, season=2023)

# Сохраняем все raw-данные в JSON
with open("fixtures.json", "w", encoding="utf-8") as f:
    json.dump(fixtures, f, ensure_ascii=False, indent=2)

In [ ]:

# Переводим в DataFrame
rows = []
for f in fixtures:
    rows.append({
        "fixture_id": f["fixture"]["id"],
        "date": f["fixture"]["date"],
        "home_team": f["teams"]["home"]["name"],
        "away_team": f["teams"]["away"]["name"],
        "home_score": f["score"]["fulltime"]["home"],
        "away_score": f["score"]["fulltime"]["away"]
    })

df = pd.DataFrame(rows)

print(df.head())
print(f"Всего матчей: {len(df)}")


In [40]:
def get_conn_to_pg():
    from dotenv import load_dotenv
    load_dotenv()
    return psycopg2.connect(
        database=os.environ['PG_DB'],
        user=os.environ['PG_USR'],
        password=os.environ['PG_PWD'],
        host=os.environ['PG_HOST'],
        port=os.environ['PG_PORT']
    )


In [84]:
# Load data from JSON file
def load_leagues_to_db(file_name):
    with open(file_name, "r", encoding="utf-8") as f:
        data_league = json.load(f)

    # Connect to PostgreSQL
    conn = get_conn_to_pg()
    cur = conn.cursor()

    countries = set()
    for league in data_league['response']:
        country = league['country']['name']
        code = league['country']['code']
        flag = league['country']['flag']
        countries.add((country, code, flag))

    countries_with_id = [
        (idx, name, code, flag)
        for idx, (name, code, flag) in enumerate(sorted(list(countries), key=lambda x: x[0]), start=1)
    ]
    created_at = datetime.now().isoformat()
    updated_at = datetime.now().isoformat()

    for country in countries_with_id:
        id = country[0]
        name = country[1]
        code = country[2]
        flag_image = country[3]
        # print(code)
        cur.execute(
            "INSERT INTO public.football_app_country (id, name, code, flag_image, created_at, updated_at) VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT (id) DO NOTHING",
            (id, name, code, flag_image, created_at, updated_at)
        )

    for id, league in enumerate(data_league['response']):
        external_id = league['league']['id']
        league_name = league['league']['name']
        country = league['country']['name']
        logo_image = league['league']['logo']
        type = league['league']['type']
        country_id = next((c[0] for c in countries_with_id if c[1] == country), None)
        is_active = False
        cur.execute(
            "INSERT INTO public.football_app_league (id, name, logo_image, is_active, country_id, external_id, type, created_at, updated_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (id) DO NOTHING",
            (id, league_name, logo_image, is_active, country_id, external_id, type, created_at, updated_at)
        )

    conn.commit()
    cur.close()
    conn.close()

In [85]:
file_name = "leagues_response.json"
load_leagues_to_db(file_name)

In [74]:
with open("leagues_response.json", "r", encoding="utf-8") as f:
    data_league = json.load(f)

countries = set()
for league in data_league['response']:
    country = league['country']['name']
    code = league['country']['code']
    flag = league['country']['flag']
    countries.add((country, code, flag))

countries_with_id = [
    (idx, name, code, flag)
    for idx, (name, code, flag) in enumerate(sorted(list(countries), key=lambda x: x[0]), start=1)
]

In [77]:
from collections import Counter

counts = Counter(c[2] for c in countries_with_id)
# print(counts)
duplicates = [c for c in countries_with_id if counts[c[2]] > 1]
print(duplicates)


[(38, 'Crimea', 'UA', 'https://media.api-sports.io/flags/ua.svg'), (161, 'Ukraine', 'UA', 'https://media.api-sports.io/flags/ua.svg')]


In [69]:
 for id, league in enumerate(data_league['response']):
    print(f"League id = {league['league']['id']}: {league['league']['name']}, Country: {league['country']['name']}")

League id = 4: Euro Championship, Country: World
League id = 21: Confederations Cup, Country: World
League id = 62: Ligue 2, Country: France
League id = 61: Ligue 1, Country: France
League id = 144: Jupiler Pro League, Country: Belgium
League id = 71: Serie A, Country: Brazil
League id = 39: Premier League, Country: England
League id = 78: Bundesliga, Country: Germany
League id = 135: Serie A, Country: Italy
League id = 88: Eredivisie, Country: Netherlands
League id = 94: Primeira Liga, Country: Portugal
League id = 140: La Liga, Country: Spain
League id = 179: Premiership, Country: Scotland
League id = 180: Championship, Country: Scotland
League id = 1: World Cup, Country: World
League id = 803: Asian Games, Country: World
League id = 804: Caribbean Cup, Country: World
League id = 40: Championship, Country: England
League id = 46: EFL Trophy, Country: England
League id = 45: FA Cup, Country: England
League id = 47: FA Trophy, Country: England
League id = 48: League Cup, Country: Engla

In [48]:
len(countries_dict)
countries_dict['England']

7